In [ ]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2
%matplotlib ipympl


import numpy as np
from utils import Lmatrix2paths, adapted_empirical_measure, adapted_wasserstein_squared, quantization, nested, plot_V

verbose = False

random_seed = np.random.randint(100)
random_seed

# Same randomness

In [3]:
n_sample = 1000
normalize = False

print("mu")
L = np.array([[1, 0, 0], [2, 4, 0], [3, 2, 1]])
X,A = Lmatrix2paths(L, n_sample, seed = random_seed)

print("nu")
M = np.array([[1, 0, 0], [2, 3, 0], [3, 1, 2]])
Y,B = Lmatrix2paths(M, n_sample, seed = random_seed)

# Grid projection of k-mean projection
adaptedX = adapted_empirical_measure(X, delta_n = 0.1)
adaptedY = adapted_empirical_measure(Y, delta_n = 0.1)

q2v, v2q, mu_x, nu_y, q2v_x, v2q_x, q2v_y, v2q_y = quantization(adaptedX, adaptedY, markovian=False)
AW_2square, V = nested(mu_x, nu_y, v2q_x, v2q_y, q2v, markovian=False)
dist_bench = adapted_wasserstein_squared(A, B)
print("Theoretical AW_2^2: ", dist_bench)
print("Numerical AW_2^2: ", AW_2square)

mu
Cholesky:
[[1 0 0]
 [2 4 0]
 [3 2 1]]
Covariance:
[[ 1  2  3]
 [ 2 20 14]
 [ 3 14 14]]
nu
Cholesky:
[[1 0 0]
 [2 3 0]
 [3 1 2]]
Covariance:
[[ 1  2  3]
 [ 2 13  9]
 [ 3  9 14]]
Quantization ......
Number of distinct values in global quantization:  240
Number of condition subpaths of mu_x
Time 0: 1
Time 1: 60
Time 2: 921
Number of condition subpaths of nu_y
Time 0: 1
Time 1: 60
Time 2: 884
Nested backward induction .......


Timestep 0: 100%|██████████| 1/1 [00:00<00:00, 869.47it/s]

Theoretical AW_2^2:  3.0
Numerical AW_2^2:  3.1208666666666667


# Different Randomness

In [4]:
n_sample = 1000
normalize = False

print("mu")
L = np.array([[1, 0, 0], [2, 4, 0], [3, 2, 1]])
X,A = Lmatrix2paths(L, n_sample, seed = random_seed)

print("nu")
M = np.array([[1, 0, 0], [2, 3, 0], [3, 1, 2]])
Y,B = Lmatrix2paths(M, n_sample, seed = random_seed+1)

# Grid projection of k-mean projection
adaptedX = adapted_empirical_measure(X, delta_n = 0.1)
adaptedY = adapted_empirical_measure(Y, delta_n = 0.1)

q2v, v2q, mu_x, nu_y, q2v_x, v2q_x, q2v_y, v2q_y = quantization(adaptedX, adaptedY, markovian=False)
AW_2square, V = nested(mu_x, nu_y, v2q_x, v2q_y, q2v, markovian=False)
dist_bench = adapted_wasserstein_squared(A, B)
print("Theoretical AW_2^2: ", dist_bench)
print("Numerical AW_2^2: ", AW_2square)

mu
Cholesky:
[[1 0 0]
 [2 4 0]
 [3 2 1]]
Covariance:
[[ 1  2  3]
 [ 2 20 14]
 [ 3 14 14]]
nu
Cholesky:
[[1 0 0]
 [2 3 0]
 [3 1 2]]
Covariance:
[[ 1  2  3]
 [ 2 13  9]
 [ 3  9 14]]
Quantization ......
Number of distinct values in global quantization:  234
Number of condition subpaths of mu_x
Time 0: 1
Time 1: 60
Time 2: 921
Number of condition subpaths of nu_y
Time 0: 1
Time 1: 57
Time 2: 879
Nested backward induction .......


Timestep 0: 100%|██████████| 1/1 [00:00<00:00, 799.37it/s]

Theoretical AW_2^2:  3.0
Numerical AW_2^2:  6.053913891755634


In [ ]:
for k1, v1 in mu_x[t].items():
    for k2, v2 in nu_y[t].items():
        # list of probability of conditional distribution mu_x
        w1 = list(v1.values())
        w1 = np.array(w1) / sum(w1)
        # list of probability of conditional distribution nu_y
        w2 = list(v2.values())
        w2 = np.array(w2) / sum(w2)
        # list of quantized values of conditional distribution mu_x (nu_y)
        q1 = list(v1.keys())
        q2 = list(v2.keys())
        # square cost of the values indexed by quantized values: |q2v[q1] - q2v[q2]|^2
        cost = square_cost_matrix[np.ix_(q1, q2)]

        # At T-1: add V[T] = 0, otherwise add the V[t+1] already computed
        if t < T - 1:
            q1s = [v2q_x[t + 1][k1 + (q,)] for q in v1.keys()]
            q2s = [v2q_y[t + 1][k2 + (q,)] for q in v2.keys()]
            cost += V[t + 1][np.ix_(q1s, q2s)]

        # solve the OT problem with cost |x_t-y_t|^2 + V_{t+1}(x_{1:t},y_{1:t})
        V[t][v2q_x[t][k1], v2q_y[t][k2]] = ot.emd2(w1, w2, cost)